In [148]:
import cv2
import h5py
import numpy as np
import random

In [149]:
train_data = r'C:\Users\vrinda\Documents\GitHub\computer-vision-hands-on\Problem\dataset\wood_species_train.hdF5'
with h5py.File(train_data) as f:
    imgs = f['images'][:]
    labels = f['labels'][:]
    
print(imgs.shape,labels.shape)
print(imgs)

(5708, 200, 200, 3) (5708,)
[[[[180 177 169]
   [187 184 176]
   [193 190 182]
   ...
   [162 159 155]
   [166 163 159]
   [172 169 165]]

  [[179 176 168]
   [185 182 174]
   [191 188 180]
   ...
   [163 160 156]
   [167 164 160]
   [173 170 166]]

  [[178 175 167]
   [182 179 171]
   [187 184 176]
   ...
   [165 162 158]
   [168 165 161]
   [174 171 167]]

  ...

  [[182 179 171]
   [177 174 166]
   [173 170 162]
   ...
   [177 175 165]
   [173 171 161]
   [171 169 159]]

  [[178 175 167]
   [174 171 163]
   [172 169 161]
   ...
   [176 174 164]
   [174 172 162]
   [174 172 162]]

  [[173 170 162]
   [171 168 160]
   [170 167 159]
   ...
   [176 174 164]
   [175 173 163]
   [178 176 166]]]


 [[[163 161 161]
   [162 160 160]
   [161 159 159]
   ...
   [186 178 165]
   [190 181 168]
   [193 184 171]]

  [[162 160 160]
   [161 159 159]
   [161 159 159]
   ...
   [186 178 165]
   [189 180 167]
   [191 182 169]]

  [[161 159 159]
   [161 159 159]
   [161 159 159]
   ...
   [186 178 165]


In [150]:
from sklearn.model_selection import train_test_split 
(trainX, testX, trainY, testY) = train_test_split(imgs,labels, test_size=0.25) 

In [151]:
orb = cv2.ORB_create()
trainX_descriptors = []
trainY=[]
for i, img in enumerate(trainX):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    key, desc = orb.detectAndCompute(image, None)
    if desc is not None:
        trainX_descriptors.append(desc) 
        trainY.append(labels[i])

print(len(trainX_descriptors))
print(len(trainY))

3609
3609


In [152]:
fitX_descriptors = np.vstack(trainX_descriptors)

In [153]:
from sklearn.cluster import KMeans 
kmeans = KMeans(n_clusters =10,init = 'k-means++')
kmeans.fit(fitX_descriptors)

c:\Users\vrinda\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10)

In [154]:
print(len(trainX_descriptors))

3609


In [155]:

hist_list =[]
for i in trainX_descriptors:
    PRED = kmeans.predict(i)
    histogram, _ = np.histogram(PRED, bins=10)
    hist_list.append(histogram)
print("Histogram:", hist_list)   
print(i.shape)    

Histogram: [array([3, 2, 1, 2, 4, 0, 0, 1, 1, 2], dtype=int64), array([ 6,  0,  4,  6, 11,  0,  3,  1,  0, 18], dtype=int64), array([ 3,  4, 10,  3,  1,  3,  6,  4,  2,  8], dtype=int64), array([2, 0, 0, 0, 1, 2, 0, 1, 4, 2], dtype=int64), array([ 7, 19,  7,  3,  8,  9, 11, 10,  8,  7], dtype=int64), array([2, 0, 0, 0, 1, 1, 0, 0, 0, 2], dtype=int64), array([20, 22, 22,  9, 21,  8, 37, 18, 21, 18], dtype=int64), array([ 5,  0,  5,  3,  4,  0, 17,  2,  6,  1], dtype=int64), array([1, 2, 1, 0, 0, 0, 0, 4, 2, 1], dtype=int64), array([20, 12, 19, 15, 24,  7, 40, 15, 15,  7], dtype=int64), array([15,  7, 12,  9, 12,  4, 33, 11, 14,  8], dtype=int64), array([34, 41, 29, 27, 17, 69, 22, 46, 38, 11], dtype=int64), array([14,  9, 11, 20,  8, 17, 16, 16, 15, 14], dtype=int64), array([32, 46, 17, 42, 22, 67, 37, 44, 25, 39], dtype=int64), array([2, 4, 1, 3, 5, 4, 3, 3, 4, 2], dtype=int64), array([ 7,  7, 11, 14,  6, 18, 10, 16,  5, 13], dtype=int64), array([1, 0, 0, 0, 2, 0, 0, 0, 1, 1], dtype=in

In [156]:
# from sklearn.preprocessing import StandardScaler
# stdslr=StandardScaler().fit(hist_list)
# hist_list=stdslr.transform(hist_list)

In [157]:
print(len(hist_list))

3609


In [158]:
from sklearn.svm import LinearSVC
clf=LinearSVC(max_iter=200000)
clf.fit(hist_list,trainY)

c:\Users\vrinda\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=200000)

In [159]:

testX_descriptors = []
testY=[]
for i, img in enumerate(testX):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    key, desc = orb.detectAndCompute(image, None)
    if desc is not None:
        testX_descriptors.append(desc) 
        testY.append(labels[i])

print(len(testX_descriptors))
print(len(testY))

1221
1221


In [160]:
hist_test =[]
for i in testX_descriptors:
    PRED = kmeans.predict(i)
    histogram, _ = np.histogram(PRED, bins=10)
    hist_test.append(histogram)
print("Histogram:", hist_test)   
 

Histogram: [array([2, 0, 0, 0, 0, 0, 1, 0, 0, 1], dtype=int64), array([ 5, 12,  7, 12,  9,  9,  4, 17, 10, 13], dtype=int64), array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64), array([24, 21, 21, 24, 27, 12, 15, 15, 10, 18], dtype=int64), array([26, 16, 25, 24, 22, 19, 10, 19, 26, 30], dtype=int64), array([3, 0, 0, 6, 0, 1, 4, 5, 2, 7], dtype=int64), array([34, 22, 35, 23, 44, 18, 50, 30, 40, 43], dtype=int64), array([2, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=int64), array([45, 27, 31, 36, 62,  5, 42, 25, 34, 24], dtype=int64), array([47, 40, 20, 30, 32, 54, 39, 36, 62, 33], dtype=int64), array([5, 1, 0, 6, 4, 2, 6, 0, 2, 2], dtype=int64), array([13,  2,  5, 26, 11,  1, 27,  7,  7, 11], dtype=int64), array([1, 3, 0, 1, 0, 3, 0, 1, 0, 4], dtype=int64), array([4, 1, 0, 2, 0, 0, 3, 0, 0, 2], dtype=int64), array([10,  6,  5,  3,  2,  8, 10,  6, 10,  3], dtype=int64), array([13,  9, 18,  4,  8,  4,  7,  5,  5, 10], dtype=int64), array([1, 2, 0, 2, 0, 1, 1, 0, 1, 1], dtype=int64), array([6, 2, 5

In [161]:
# hist_test=stdslr.transform(hist_test)


In [162]:
TEST= clf.predict(hist_test)
print(len(TEST))

1221


In [163]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(testY,TEST)
print(accuracy*100)

12.039312039312039
